## Asset symbols

In [150]:
Gold = 'GC=F'
Volatility_Index = '^VIX'
BTC = 'BTC-USD'
ETH = 'ETH-USD'
ADA = 'ADA-USD'

## Import Packages

In [151]:
from pandas_datareader import data as web
from datetime import date
import math
from forex_python.converter import CurrencyRates

c = CurrencyRates()

## Define Dates

In [101]:
# Dates should be in the format 'yyyy/mm/dd'
sdate = '2019/07/05'

# Get todays date
edate = date.today()

## Given some amount of cash £X, what is the most of Stock Y that can be bought at its current price, given a worst case scenario stock price of Z?

In [143]:
# This assumes the currency used to purchase the asset is £GBP and the asset being purchased is in $USD
def max_shares(amount, ticker, worst_case_scenario):    
    
    # Reads data into a dataframe
    df = web.DataReader(ticker, 'yahoo', sdate, edate)
    
    # Get live conversion rate GBP:USD
    conversion_rate = round(c.get_rate('GBP', 'USD'), 2)
    # If forex_python is making the script slow then uncomment the line blow,
    # comment out the line above and manually adjust
    
    # conversion_rate = 1.2

    # Calculate the dollar amount using the conversion rate
    dollar_amount = math.floor(conversion_rate * amount)
   
    # We define the current price as the latest closing price
    price = round(df['Close'][-1], 2)
    
    # Calculate the amount required to hold a single share
    maintenance_per_share = price / 10
    
    # Calculate the maximum number of shares we can buy based on the worst_case_scenrio
    constraint_1 = math.floor((dollar_amount/(price - worst_case_scenario + maintenance_per_share)) * 10) / 10.0
    
    # Calculate the maximum number of shares we can buy based on the margin required to purchase the shares
    constraint_2 = math.floor((5 * dollar_amount / price)* 10) / 10.0

    # The maximum amount of shares we can buy is the minimum of the 2 constraints above
    max_shares = min(constraint_1, constraint_2)
    
    # Calculate the required margin to hold the calculated amount of shares
    required_margin = math.ceil((price / 5) * max_shares)
    maintenance_margin = math.ceil(required_margin / 2)
    
    # We define the highest price as the highest price in the period
    period_high = round(max(df['High']), 2)
    
    # Calculate the profit if the stock went back to its period high
    period_high_profit = round((period_high - price) * max_shares, 0)
    
    # Calculate return on investment (ROI) if the stock reaches period high
    roi = round((period_high_profit / dollar_amount) * 100, 2)
    
    # Print info
    print(f"With your £{amount}, your equity will be ${dollar_amount}.")
    print(f"The current {ticker} share price is ${price}.")
    print(f"You can buy {max_shares} shares but your positions will close out if {ticker} reaches ${worst_case_scenario}.")
    print("")
    print(f"The required margin for {max_shares} shares is ${required_margin}")
    print(f"The maintenance margin is ${maintenance_margin}.")
    print(f"To ensure your positions don't close out, your equity must exceed ${maintenance_margin}.")
    print("")
    print(f"The period high for {ticker} is ${period_high}.")
    print(f"If {ticker} reaches ${period_high} again, profit will be ${period_high_profit}.")
    print(f"Your equity will be ${dollar_amount + period_high_profit}.")
    print(f"Return on investment would be {roi}%")
    

In [146]:
max_shares(100, 'TSLA',  550)

With your £100, your equity will be $120.
The current TSLA share price is $752.29.
You can buy 0.4 shares but your positions will close out if TSLA reaches $550.

The required margin for 0.4 shares is $61
The maintenance margin is $31.
To ensure your positions don't close out, your equity must exceed $31.

The period high for TSLA is $1243.49.
If TSLA reaches $1243.49 again, profit will be $196.0.
Your equity will be $316.0.
Return on investment would be 163.33%


In [145]:
max_shares(20, 'SPCE', 5)

With your £20, your equity will be $24.
The current SPCE share price is $7.32.
You can buy 7.8 shares but your positions will close out if SPCE reaches $5.

The required margin for 7.8 shares is $12
The maintenance margin is $6.
To ensure your positions don't close out, your equity must exceed $6.

The period high for SPCE is $62.8.
If SPCE reaches $62.8 again, profit will be $433.0.
Your equity will be $457.0.
Return on investment would be 1804.17%


In [142]:
max_shares(120, 'NIO', 10)

With your £120, your equity will be $144.
The current NIO share price is $22.6.
You can buy 9.6 shares but your positions will close out if NIO reaches $10.

The required margin for 9.6 shares is $44
The maintenance margin is $22.
To ensure your positions don't close out, your equity must exceed $22.

The period high for NIO is $66.99.
If NIO reaches $66.99 again, profit will be $426.0.
Your equity will be $570.0.
Return on investment would be 295.83%


In [148]:
max_shares(1200, Gold, 1500)

With your £1200, your equity will be $1440.
The current GC=F share price is $1740.6.
You can buy 3.4 shares but your positions will close out if GC=F reaches $1500.

The required margin for 3.4 shares is $1184
The maintenance margin is $592.
To ensure your positions don't close out, your equity must exceed $592.

The period high for GC=F is $2072.0.
If GC=F reaches $2072.0 again, profit will be $1127.0.
Your equity will be $2567.0.
Return on investment would be 78.26%


In [152]:
max_shares(2000, BTC, 10000)

With your £2000, your equity will be $2400.
The current BTC-USD share price is $20928.46.
You can buy 0.1 shares but your positions will close out if BTC-USD reaches $10000.

The required margin for 0.1 shares is $419
The maintenance margin is $210.
To ensure your positions don't close out, your equity must exceed $210.

The period high for BTC-USD is $68789.62.
If BTC-USD reaches $68789.62 again, profit will be $4786.0.
Your equity will be $7186.0.
Return on investment would be 199.42%
